<a href="https://colab.research.google.com/github/1900690/yolo-traking/blob/main/yolo_traking_panorama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#果実トラッキング

※学習前のＡＩで追跡を行う。追跡できる品目はリンゴ、オレンジなど代表的なもののみ

In [1]:
#@title 分析する動画をアップロードまたはサンプル動画(ユズ)で分析

#事前準備
!pip -q install ultralytics

from google.colab import files
import os
サンプル動画を使用する = False # @param {type:"boolean"}

if サンプル動画を使用する == True:
  if os.path.exists('/content/yuzu.mp4'):
     os.remove('/content/yuzu.mp4')
  !wget -q  https://github.com/1900690/yolo-traking/releases/download/sample/yuzu.mp4

else:
  uploaded_file = files.upload()
  uploaded_file_name = next(iter(uploaded_file))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.3/887.3 kB 13.7 MB/s eta 0:00:00


Saving 60fpsリンゴ.mp4 to 60fpsリンゴ.mp4


<p align="center">
  <img src="https://github.com/1900690/yolo-traking/blob/main/images/yuzu_before.jpg?raw=1" width=80%>
  <br>
  ユズのサンプル動画
</p>

In [ ]:
#@markdown 分析して動画をダウンロード
#因数は以下を参照https://docs.ultralytics.com/modes/predict/#inference-arguments
from ultralytics import YOLO
import shutil
import os
from google.colab import files

if os.path.exists('/content/runs'):
    shutil.rmtree('/content/runs')

# Load a model
model = YOLO("yolo11x.pt")

if サンプル動画を使用する == True:
  results = model.track(source="/content/yuzu.mp4",stream=False,save=True)
  files.download("/content/runs/detect/track/yuzu.avi")
else:
  results = model.track(source="/content/"+uploaded_file_name,stream=False,save=True,classes=47,save_txt=True,show_conf=False)
  files.download("/content/runs/detect/track/"+uploaded_file_name[:-4]+".avi")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 109M/109M [00:00<00:00, 187MB/s] 


requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.4s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1115) /content/60fpsリンゴ (1).mp4: 384x640 22 apples, 3415.9ms
video 1/1 (frame 2/1115) /content/60fpsリンゴ (1).mp4: 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<p align="center">
  <img src="https://github.com/1900690/yolo-traking/blob/main/images/yuzu_after.jpg?raw=1" width=80%>
  <br>
  果実数を検出後のユズのサンプル動画
</p>

特定の範囲を通り過ぎたものの数をカウントする

In [2]:
import cv2

from ultralytics import solutions

cap = cv2.VideoCapture("/content/60fpsリンゴ.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define region points
#通り過ぎた線上の物体をカウントする
region_points = [(640, 0), (640, 720)]  # For line counting
#フレームいっぱいを検知範囲にすればカウントできるのでは？
#region_points = [(0, 720), (1280, 720), (1280, 0), (0, 0)]  # For rectangle region counting
# region_points = [(20, 400), (1080, 404), (1080, 360), (20, 360), (20, 400)]  # For polygon region counting

# Video writer
video_writer = cv2.VideoWriter("/content/60fpsリンゴ線カウント.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    #show=True,  # Display the output
    region=region_points,  # Pass region points
    model="yolo11x.pt",  # model="yolo11n-obb.pt" for object counting using YOLO11 OBB model.
    classes=[47],  # If you want to count specific classes i.e person and car with COCO pretrained model.
    show_in=False,  # Display in counts
    # show_out=True,  # Display out counts
    # line_width=2,  # Adjust the line width for bounding boxes and text display
)

# Process video
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    im0 = counter.count(im0)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
requirements: Ultralytics requirement ['shapely>=2.0.0'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.1s, installed 1 package: ['shapely>=2.0.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Ultralytics Solutions: ✅ {'region': [(640, 0), (640, 720)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'model': 'yolo11x.pt', 'classes': [47]}


100%|██████████| 109M/109M [00:00<00:00, 196MB/s] 


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 1.8s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 23 apples, 312.3ms
Speed: 2.7ms preprocess, 312.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 apples, 274.3ms
Speed: 1.6ms preprocess, 274.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 apples, 323.8ms
Speed: 1.7ms preprocess, 323.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 apples, 267.2ms
Speed: 1.7ms preprocess, 267.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 apples, 272.3ms
Speed: 1.7ms preprocess, 272.3ms inference, 1.1ms postp

パノラマ画像を作って、それを左から右に流してカウントするのがよさそう
[OpenCVで動画フレームからパノラマ画像の生成](https://kdl-di.hatenablog.com/entry/2022/07/08/113500)

In [ ]:
import cv2 # OpenCVの呼び出し

video = cv2.VideoCapture('path/to/video.mp4') # 動画の読み込み

while True:
    ret, frame = video.read() #1フレームずつ取り出す
    if ret == False:          #動画が終わっている場合は終了する
        break
    cv2.imshow("test",frame)  #取り出したフレームを表示する
    cv2.waitKey(int(1 / video.get(cv2.CAP_PROP_FPS) * 1000))#撮影時のフレームレートを自動で認識し、それに応じて待機。（60FPSなら1/60）
cv2.destroyAllWindows() #ウィンドウを閉じる

In [ ]:
import os   #osというライブラリを呼び出す
os.makedirs("./cut_img",exist_ok=True)
video = cv2.VideoCapture('./movie.MOV') # 動画の読み込み

skip_frame = 30 #スキップするフレーム数
frame_num = 0
counter =0
while True:
    ret, frame = video.read() #1フレームずつ取り出す
    if ret == False:          #動画が終わっている場合は終了する
        break
    if frame_num % skip_frame==0: #スキップ数に達すると保存する
        cv2.imwrite(f"./cut_img/{counter:05}.jpg",frame)
        counter += 1
    frame_num += 1

In [ ]:
import glob
img_list = glob.glob(os.path.join("./cut_img/","*.jpg"))

imgs = [cv2.imread(path) for path in img_list]

stitcher = cv2.Stitcher_create()

result = stitcher.stitch(imgs)[1]
cv2.imwrite("Panorama.jpg",result)

パノラマを動画にするには[iMOVE](https://x.com/baru_tozan/status/1812038440461930554)を使う方法